In [1]:
import os
from flask import Flask
import json
import time
import auth
import pytz
import socket
import sqlite3
import numpy as np
import datetime
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from pandas.io.json import json_normalize
from flask_sqlalchemy import SQLAlchemy
pd.set_option('display.max_colwidth', -1)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def get_datax(_db):
    conn = sqlite3.connect(_db)
    cur = conn.cursor()
    cur.execute("SELECT json_data FROM flight_data limit 1")
    _data = cur.fetchall()
    cur.close()
    conn.close()
    return _data

def determine_direction(x):
    """

    :param x:
    :return:
    """
    x = str(x)
    if x == "BTS" or x == "KSC" or x == "SLD" or x == "TAT":
        return str("TAM")
    else:
        return str("SPAT")


def determine_production(x):
    """

    :param x:
    :return:
    """
    x = str(x)
    if x == "BTS":
        return str("BTS")
    elif x == "KSC":
        return str("KSC")
    else:
        return str("---")


def extra_catering(x):
    """

    :param x:
    :return:
    """
    try:
        return ['{0}: {1}'.format(i['code'],i['count']) for i in x]
    except:
        return 0


def split_weird_timeframes(_i, _dataframe):
    """

    :param _i:
    :param _dataframe:
    :return:
    """
    _s = _i.split('___')[0]
    _e = _i.split('___')[1]
    return _dataframe.loc[_s:_e]


def add_one_day(_ii):
    _plus_day = datetime.datetime.strptime(_ii, '%Y-%m-%d %H:%M:%S') + datetime.timedelta(1)
    return _plus_day



def get_unique_days(_data):
    """

    :param _data:
    :return:
    """
    _raw_data = json.loads((_data[0][0]).decode("utf-8"))
    _df = json_normalize(_raw_data['data']['flight']['data'])
    
    # _df = json_normalize(_data['data']['flight']['data'])
    _df_normalized = _df
    
    _all_unique_days = _df['local_std_date'].unique()
    _all_unique_days = np.sort(_all_unique_days)
    _all_unique_days = ['{}___{}'.format(_i + ' 09:00:00', add_one_day(_i + ' 09:00:00')) for _i in _all_unique_days]
    _all_unique_days[0] = str(datetime.datetime.strptime(_all_unique_days[0].split('___')[0], '%Y-%m-%d %H:%M:%S') \
                          - datetime.timedelta(hours=9)) + "___" + str(_all_unique_days[0].split('___')[1])
    print(_all_unique_days)
    return _all_unique_days, _df_normalized

def render_tables(_df_normalized):
    """

    :param _df_normalized:
    :return:
    """
    _df = _df_normalized
    _compare = pd.DataFrame()

    _compare['Departure'] = pd.to_datetime(_df['local_std_date'] + ' ' + _df['local_std_time'])
    _compare['Depart'] = _compare['Departure']
    _compare['Arrival'] = pd.to_datetime(_df['local_sta_date'] + ' ' + _df['local_sta_time'])
    _compare['Flight'] = _df['flight_number']
    _compare['Aircraft'] = _df['aircraft_config']
    _compare['Reg'] = _df['aircraft_reg']
    _compare['Route'] = _df['route_id']
    _compare["Meal"] = _df["catering_order.flight_meal_type"]
    _compare['Direction'] = _df['departure_iata'].map(determine_direction)
    _compare['Production'] = _df['departure_iata'].map(determine_production)
    _compare['From'] = _df['departure_iata']
    _compare['To'] = _df['destination_iata']

    _compare['Quantity'] = _df['catering_order.quantity_y']
    _compare['Crew'] = _df['catering_order.quantity_crew']
    _compare['Extra Catering'] = _df['extra_catering'].map(extra_catering)
    _compare['Note'] = _df['catering_order.general_note'].map(
        lambda x: "{0}{1}{2}".format("""<div class="hoverable">""", str(x), "</div>"))
    _compare['Note'] = _compare['Note'].map(lambda x: str(x).replace('\n', "<br>"))
    _compare['Note'] = _compare['Note'].map(lambda x: str(x).replace('\r', ''))
    _compare['Quantity'] = _compare['Quantity'].fillna(0)
    _compare['Crew'] = _compare['Crew'].fillna(0)
    _compare.index = _compare['Departure']
    return _compare



In [11]:
db = "2w.sqlite"
data = get_datax(db)
ud, normal = get_unique_days(data)
compare = render_tables(normal)
compare = compare.sort_values(['Depart'], ascending=[True])

['2018-07-17 00:00:00___2018-07-18 09:00:00', '2018-07-18 09:00:00___2018-07-19 09:00:00', '2018-07-19 09:00:00___2018-07-20 09:00:00', '2018-07-20 09:00:00___2018-07-21 09:00:00', '2018-07-21 09:00:00___2018-07-22 09:00:00']


In [111]:
def get_unique_days(_data):
    """
    :param _data:
    :return:
    """
    _raw_data = json.loads((_data[0][0]).decode("utf-8"))
    _df = json_normalize(_raw_data['data']['flight']['data'])
    
    grouped = _df.groupby(['local_std_date'])
    
    _all_unique_days = []
    for day, group in grouped:
        minimum = group['local_std_time'].min()
        maximum = group['local_std_time'].max()
        _all_unique_days.append((day, minimum, maximum))
        
    start, end = _all_unique_days[::len(_all_unique_days) - 1]
    start = list(start[:2])
    end = list(end[::len(end)-1])
    start = datetime.datetime.strptime(start[0] + ' ' + start[1], '%Y-%m-%d %H:%M:%S')
    end = datetime.datetime.strptime(end[0] + ' ' + end[1], '%Y-%m-%d %H:%M:%S')
    final = []
    a = start
    while True:
        b = (a + datetime.timedelta(1)).replace(hour=9, minute=0)
        if b > end:
            b = end
            final.append('{}___{}'.format(str(a), str(b)))
            break
        final.append('{}___{}'.format(str(a), str(b)))
        a = b
    return final, _df
    
get_unique_days(data)
    

(['2018-07-17 02:45:00___2018-07-18 09:00:00',
  '2018-07-18 09:00:00___2018-07-19 09:00:00',
  '2018-07-19 09:00:00___2018-07-20 09:00:00',
  '2018-07-20 09:00:00___2018-07-21 01:25:00'],
     aircraft_config aircraft_reg aircraft_type  catering_order  \
 0    189Y            OM-GTF       73B          NaN               
 1    189Y            OM-GTF       73B          NaN               
 2    189Y            OM-GTF       73B          NaN               
 3    189Y            OM-GTF       73B          NaN               
 4    189Y            OM-GTF       73B          NaN               
 5    189Y            OM-GTF       73B          NaN               
 6    189Y            OM-GTF       73B          NaN               
 7    189Y            OM-GTF       73B          NaN               
 8    189Y            OM-GEX       73A          NaN               
 9    189Y            OM-GEX       73A          NaN               
 10   189Y            OM-GEX       73A          NaN               
 11   1

In [75]:
str(str_end)

'2018-07-21 01:25:00'

In [3]:
_day_dict_lookup = {i.replace(' ', '_').replace(':', '_'): i for i in ud}
# temp_table_day_chunk = split_weird_timeframes(_day_dict_lookup[DAY], compare)
_day_dict_lookup

{'2018-07-17_00_00_00___2018-07-18_09_00_00': '2018-07-17 00:00:00___2018-07-18 09:00:00',
 '2018-07-18_09_00_00___2018-07-19_09_00_00': '2018-07-18 09:00:00___2018-07-19 09:00:00',
 '2018-07-19_09_00_00___2018-07-20_09_00_00': '2018-07-19 09:00:00___2018-07-20 09:00:00',
 '2018-07-20_09_00_00___2018-07-21_09_00_00': '2018-07-20 09:00:00___2018-07-21 09:00:00',
 '2018-07-21_09_00_00___2018-07-22_09_00_00': '2018-07-21 09:00:00___2018-07-22 09:00:00'}

In [4]:
def split_data_at_time(_df, _time):
    a = _df.loc[_time:, :]
    delimiter = a.iloc[:1].index.values[0]
    b = _df.loc[:delimiter, :]
    b = b.drop(delimiter)
    return a, b, delimiter

def get_unique_routes(_df):
    return set(_df['Route'].unique().tolist())

def do(_day):
    DAY = _day_dict_lookup[_day]
    FT, ST = DAY.split('___')
    valid, invalid, dm1 = split_data_at_time(compare, FT)
    extra, main, dm2 = split_data_at_time(valid, ST)
    valid_rids = get_unique_routes(valid)
    invalid_rids = get_unique_routes(invalid)
    intersection = invalid_rids & valid_rids
    main_rids = get_unique_routes(main)
    main_rids = main_rids - intersection
    final = valid[valid['Route'].isin(list(main_rids))]
    return final

In [8]:
def do(_day):
    DAY = _day_dict_lookup[_day]
    FT, ST = DAY.split('___')
    valid, invalid, dm1 = split_data_at_time(compare, FT)
    extra, main, dm2 = split_data_at_time(valid, ST)
    valid_rids = get_unique_routes(valid)
    invalid_rids = get_unique_routes(invalid)
    intersection = invalid_rids & valid_rids
    main_rids = get_unique_routes(main)
    main_rids = main_rids - intersection
    final = valid[valid['Route'].isin(list(main_rids))]
    return final

In [9]:
final = do('2018-07-18_09_00_00___2018-07-19_09_00_00')

In [10]:
final

,Departure,Depart,Arrival,Flight,Aircraft,Reg,Route,Meal,Direction,Production,From,To,Quantity,Crew,Extra Catering,Note
Departure,,,,,,,,,,,,,,,,
2018-07-18 09:25:00,2018-07-18 09:25:00,2018-07-18 09:25:00,2018-07-18 12:55:00,TVQ6186,189Y,OM-GEX,13834814879014078,L2,TAM,BTS,BTS,AYT,189.0,0.0,0,"<div class=""hoverable"">6x crew snack (1 cpt, 5 crew)</div>"
2018-07-18 11:20:00,2018-07-18 11:20:00,2018-07-18 11:20:00,2018-07-18 13:50:00,TVS1564,189Y,OM-GTF,13826452434014078,L2,TAM,BTS,BTS,PMI,154.0,0.0,0,"<div class=""hoverable"">6x crew snack (1 cpt, 5 crew)</div>"
2018-07-18 12:20:00,2018-07-18 12:20:00,2018-07-18 12:20:00,2018-07-18 14:15:00,TVS1520,189Y,OK-TSD,13826452331214078,L2,TAM,BTS,BTS,OLB,151.0,0.0,0,"<div class=""hoverable"">6x crew snack (1 cpt, 5 crew)<br><br>Extra catering:<br>3x GFML/NLML</div>"
2018-07-18 13:40:00,2018-07-18 13:40:00,2018-07-18 13:40:00,2018-07-18 15:25:00,TVQ6187,189Y,OM-GEX,13834814879014078,L2,SPAT,---,AYT,BTS,186.0,0.0,0,"<div class=""hoverable""></div>"
2018-07-18 14:30:00,2018-07-18 14:30:00,2018-07-18 14:30:00,2018-07-18 18:00:00,TVQ6118,189Y,OM-TSG,13834814700014078,L2,TAM,BTS,BTS,AYT,189.0,0.0,[GFML: 3],"<div class=""hoverable"">6x crew snack (1 cpt, 5 crew)</div>"
2018-07-18 14:35:00,2018-07-18 14:35:00,2018-07-18 14:35:00,2018-07-18 17:35:00,TVS1665,189Y,OM-GTF,13826452434014078,L2,SPAT,---,PMI,KSC,180.0,0.0,0,"<div class=""hoverable""></div>"
2018-07-18 15:00:00,2018-07-18 15:00:00,2018-07-18 15:00:00,2018-07-18 16:50:00,TVS1521,189Y,OK-TSD,13826452331214078,L2,SPAT,---,OLB,BTS,151.0,0.0,0,"<div class=""hoverable""></div>"
2018-07-18 16:10:00,2018-07-18 16:10:00,2018-07-18 16:10:00,2018-07-18 18:55:00,TVQ6538,189Y,OM-GEX,13834840965614078,L1,TAM,BTS,BTS,BOJ,176.0,0.0,0,"<div class=""hoverable"">6x crew snack (1 cpt, 5 crew)</div>"
2018-07-18 17:40:00,2018-07-18 17:40:00,2018-07-18 17:40:00,2018-07-18 21:05:00,TVS1504,189Y,OM-GTE,13826452280414078,L2,TAM,BTS,BTS,HER,179.0,0.0,0,"<div class=""hoverable"">6x crew snack (1 cpt, 5 crew)</div>"
